# Jupyter Notebook to clean install all the contracts


### Necessary packages 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Up Environment

In [83]:
pprint(contracts)

{'addressProvider': {'SCORE': 'cxed440f1da2cf3087ac5ef12217a13f6321de0375',
                     'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/addressProvider'},
 'feeProvider': {'SCORE': 'cxede316aa65b7df13633599e67490daf1fd32b8a5',
                 'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider'},
 'lendingPool': {'SCORE': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
                 'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool'},
 'lendingPoolCore': {'SCORE': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
                     'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore'},
 'lendingPoolDataProvider': {'SCORE': 'cxa198eaf0a589adf0ab31b523d1f8912f47f6f293',
                             'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider'},
 'oICX': {'SCORE': 'cxf6e3d64db674c0edbf732462e80e2e3352280548',
          'zip': '/home/shubham/Work/ICON/openmoneymarket-mo

In [84]:
network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'contracts_20201108141815.pkl'
#contracts_20201109120425

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [85]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

In [86]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"


with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [21]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key

In [5]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [6]:
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")

78724202638114800000000000

### Importing wallet from keystore file

In [8]:
# Set to the address of the keystore file and give the password when prompted
password = getpass()
deployer_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
deployer_wallet.get_address()
icon_service.get_balance(deployer_wallet.get_address())

787640640000000000000000000

### Creating new wallet

In [ ]:
deployer_wallet = KeyWallet.create()
print(deployer_wallet.get_address())
caller_wallet = KeyWallet.create()
print(caller_wallet.get_address())

In [7]:
# Add the contracts. Copy the output from this cell the the one above to save the contract addresses.
#Run this when you need to zip the contract codes (before deploying them)
contracts=AutoVivification()

deploy = ['addressProvider', 'feeProvider', 'lendingPool', 'lendingPoolCore', 'lendingPoolDataProvider','oToken','priceOracle','sample_token']
# for directory in {"score"}:
directory=os.getcwd()
with os.scandir(directory) as it:
    for file in it:
        archive_name = directory + "/" + file.name
        if file.is_dir() and file.name in deploy:
            # make_archive(archive_name, "zip", directory, file.name)
            contracts[file.name]['zip'] = archive_name 


contracts

{'lendingPoolDataProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider'},
 'lendingPool': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool'},
 'feeProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider'},
 'lendingPoolCore': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore'},
 'oToken': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken'},
 'priceOracle': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/priceOracle'},
 'sample_token': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/sample_token'},
 'addressProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/addressProvider'}}

In [7]:


pprint(contracts)
contracts['addressProvider']['SCORE']='cxed440f1da2cf3087ac5ef12217a13f6321de0375'
contracts['feeProvider']['SCORE']='cxede316aa65b7df13633599e67490daf1fd32b8a5'
contracts['lendingPool']['SCORE']='cx35b12ca4e9c672518db3c06d92ed60d50e7d5554'
contracts['lendingPoolCore']['SCORE']='cx327a386a9f8076045679fc132a8baa9e270ca43b'
contracts['lendingPoolDataProvider']['SCORE']='cxa198eaf0a589adf0ab31b523d1f8912f47f6f293'
contracts['oToken']['SCORE']='cx4258464e395b3ef3b73a146947eb41393e8c5873'
contracts['priceOracle']['SCORE']='cx3f6477bf290eb91257b806ece614ab18b1c7be41'
contracts['sample_token']['SCORE']='cx19be1be7b4e7750863edf966faac3f42dec21025'



{'addressProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/addressProvider.zip'},
 'feeProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider.zip'},
 'lendingPool': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool.zip'},
 'lendingPoolCore': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore.zip'},
 'lendingPoolDataProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider.zip'},
 'oToken': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken.zip'},
 'priceOracle': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/priceOracle.zip'},
 'sample_token': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/sample_token.zip'}}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x9ab39507c7c33446525818978e89f4482804a75b0d3ce4200c8d19c9908ebde9', 'blockHeight': 9595050, 'blockHash': '0x8c69b81b2b790102afe88976ccd05dd93d2e4b4c83bed16bbfeca75932134cec', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cxf6e3d64db674c0edbf732462e80e2e3352280548', 'stepUsed': 1144284440, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1144284440, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

# Deploying all contracts


In [9]:
#this cell needs to run only when you need to deploy new set of contracts
for score, loc in contracts.items():
    params = {}
    print(score,loc)
    if score == "sample_token":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif score == "oToken":
        params = {"_name":"BridgeUSDInterestToken","_symbol":"oUSDb"}
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
now = datetime.utcnow()
filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
filename=network+'_'+filename
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

lendingPoolDataProvider {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
lendingPool {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
feeProvider {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
lendingPoolCore {'zip': '/home/shubham/Work/ICON/openmo

{'lendingPoolDataProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider',
  'SCORE': 'cx726e8bacc442256099dda50438c11d004215afc5'},
 'lendingPool': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool',
  'SCORE': 'cxbfd0ecb9892ca2e1ae3d0494d48f6618620b7e48'},
 'feeProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider',
  'SCORE': 'cxe408b46a010ff5f82e9e45610a0aadb4ff2b073c'},
 'lendingPoolCore': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore',
  'SCORE': 'cx3daa5c2517fdd44bdf37904e5e537eefafb4d69d'},
 'oToken': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken',
  'SCORE': 'cx2f4c43ec2c69321573e93549d89462de4a25fdbe'},
 'priceOracle': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/priceOracle',
  'SCORE': 'cx6703fdc7e0e65c5195d6dbfb59a838f76a9c19b1'},
 'sample_token': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/sample_token'

# Deploying otoken and sicx for icx

In [ ]:
params = {"_name":"ICXinterestToken","_symbol":"oICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oICX']["SCORE"] = res.get('scoreAddress', '')
contracts['oICX']["zip"] = '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken'

params = {"_name":"ICXinterestToken","_symbol":"oICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['sicx']["SCORE"] = res.get('scoreAddress', '')
contracts['sicx']["zip"] = '/home/shubham/Work/ICON/openmoneymarket-mono/score/sicx'

In [42]:
pprint(contracts)

{'addressProvider': {'SCORE': 'cxed440f1da2cf3087ac5ef12217a13f6321de0375',
                     'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/addressProvider'},
 'feeProvider': {'SCORE': 'cxede316aa65b7df13633599e67490daf1fd32b8a5',
                 'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider'},
 'lendingDataProvider': {'SCORE': {}, 'zip': {}},
 'lendingPool': {'SCORE': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
                 'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool'},
 'lendingPoolCore': {'SCORE': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
                     'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore'},
 'lendingPoolDataProvider': {'SCORE': 'cxa198eaf0a589adf0ab31b523d1f8912f47f6f293',
                             'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider'},
 'oICX': {'SCORE': 'cxf6e3d64db674c0edbf732462e80e2e3352280548',
          

# Updating a contract

In [71]:
contract = 'lendingPoolDataProvider'
update = contracts[contract]['SCORE']
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contracts[contract]["zip"]))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xacaaf4f1b92daee93e848927683e6b8f0e95e828daa4dd0d4c1ecdb852ffe415',
 'blockHeight': 9597053,
 'blockHash': '0xf55fec9988e66c5a148f16a20179139cec8ca17c04ae35d61b25b661577d1245',
 'txIndex': 1,
 'to': 'cxa198eaf0a589adf0ab31b523d1f8912f47f6f293',
 'scoreAddress': 'cxa198eaf0a589adf0ab31b523d1f8912f47f6f293',
 'stepUsed': 1716754400,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1716754400,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

# Configuring SCOREs's parameters

In [11]:
settings = [{'contract': 'lendingPool', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setUSDbAddress', 'params':{'_address': contracts['sample_token']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setSicxAddress', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 25*10**14}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sample_token']['SCORE'],'_sym':"USDb"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setOracleAddress', 'params':{'_address': contracts['priceOracle']['SCORE']}},
            {'contract': 'oToken', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'oToken', 'method': 'setReserveAddress', 'params':{'_address':contracts['sample_token']['SCORE']}},
            {'contract': 'oToken', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'oToken', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']['SCORE']}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
            {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['sample_token']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oToken']['SCORE']}},
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sample_token']['SCORE'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":f"1{'0'*16}","slopeRate1":f"4{'0'*16}","slopeRate2":f"5{'0'*17}"} ]}}]

In [12]:
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))

n while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0x36e6a812fb3737aa08fee259956b70c7fe02650427cc703ccbb8fff959c468f0', 'blockHeight': 1012033, 'blockHash': '0x9b13dc4d7dd65da0af1318a892a0ae04536189d9774699829bd285a6923659ca', 'txIndex': 1, 'to': 'cx726e8bacc442256099dda50438c11d004215afc5', 'stepUsed': 152720, 'stepPrice': 10000000000, 'cumulativeStepUsed': 152720, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Additional setup for icx reserve

In [75]:
icx_additional=[{'contract': 'lendingPool', 'method': 'setSICXAddress', 'params':{'_address': contracts['sicx']['SCORE']}},
                {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'sICX','_quote':'USD','_rate':5*10**17}},
                {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sicx']['SCORE'],'_sym':"sICX"}},
                {'contract': 'addressProvider', 'method': 'setsICX', 'params':{'_address':contracts['sicx']['SCORE']}},
                {'contract': 'addressProvider', 'method': 'setoICX', 'params':{'_address':contracts['oICX']['SCORE']}},
                {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sicx']['SCORE'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":f"0{'0'*17}","slopeRate1":f"7{'0'*16}","slopeRate2":f"3{'0'*18}"} ]}}
]

In [76]:
for sett in icx_additional:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))



Calling setSICXAddress, with parameters {'_address': 'cx3e9d38175d112cd5cf05a90080cce779f202ba27'} on the lendingPool contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x0215a93764d520997e9ac99e120682bfde65bfcf8d0570f18df4070cf1852a8c', 'blockHeight': 9597245, 'blockHash': '0x51220a1bd4245e45873367a9dd11352d2b40086b37178b46e9b4ac0581d04a72', 'txIndex': 1, 'to': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554', 'stepUsed': 145480, 'stepPrice': 10000000000, 'cumulativeStepUsed': 145480, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

## Adding USDb reserve in LendingPoolCore



In [13]:
params ={"_reserve": {"reserveAddress":contracts['sample_token']['SCORE'],"oTokenAddress":contracts['oToken']['SCORE'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"600000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"10","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x4b03f39ea710b9b3163775c33d0f0f78f8baf1329333c65e827457968a21bda6',
 'blockHeight': 1012108,
 'blockHash': '0x84b018701aa7669b8fbbb5e34149b8d55cf446f278b3500c25df51aa92a57171',
 'txIndex': 1,
 'to': 'cx3daa5c2517fdd44bdf37904e5e537eefafb4d69d',
 'stepUsed': 271600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 271600,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Adding ICX reserve

In [52]:
params ={"_reserve": {"reserveAddress":contracts['sicx']['SCORE'],"oTokenAddress":contracts['oICX']['SCORE'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"330000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"10","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x22c1ac8592eb4476558f90be03111f1f28d4baff5e8b69c92b82c07a34f0cc9c',
 'blockHeight': 9595709,
 'blockHash': '0x52699f953a911c93acf4808a25e2c91988efa2cb61c7c3adb412506220d14347',
 'txIndex': 1,
 'to': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
 'stepUsed': 271360,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 271360,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Transfering USDb to a wallet

In [ ]:
params = {"_to": "", "_value": 1000000000000000000000,}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Depositing USDb from a wallet

In [18]:
data = "{\"method\": \"deposit\", \"params\": {\"amount\": 1000000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 1000000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xa1e71e86c7dddeb51aa314f1e7d5414d0e677ed93f2da5bd7d1249f2a954c756',
 'blockHeight': 1460,
 'blockHash': '0xb62720a9917946e9bfb9e7e2b5b1fe59247e1b865e9ca674d0c2102141c26f6d',
 'txIndex': 1,
 'to': 'cx1592d568457a2881e4bd36117a8295ac2a000377',
 'stepUsed': 769320,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 769320,
 'eventLogs': [{'scoreAddress': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x3635c9adc5dea00000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 593',
    '0x3635c9adc5dea00000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26',
   'indexed': ['PrintData(str,int,int,int)',
    'Utilization rate core line 589',
    '0x0',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cxd2efb802be81bdca98f3024a35c28

# Borrowing USDb

In [43]:
params ={"_reserve": contracts['sample_token']['SCORE'], "_amount":10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx288685685cf0a8b8f7653c00f6dc42c8406f0d5d', 'stepLimit': '0x14bbf8', 'timestamp': '0x5b46dd9bf37cd', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'borrow', 'params': {'_reserve': 'cx1592d568457a2881e4bd36117a8295ac2a000377', '_amount': '0x8ac7230489e80000'}}, 'signature': 'Folqgh9js3lLO0q/dtvGledD0lxOz/LirI6qn2i7J4AWTIR7p2T4cxqoKL4lfDXYE9q5cEPlAjdSF6pWXzwJlgA='}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x8dd5715179fb035028e00749c33effa1d3efd06c4955ec71504df6cc145dd53b',
 'blockHeight': 4799,
 'blockHash': '0xbce5c543a5e8b5bd12ed8adf8606a2b135bbe0638fd69bfa816dcddc3651ead2',
 'txIndex': 1,
 'to': 'cx288685685cf0a8b8f7653c00f6dc42c8406f0d5d',
 'stepUsed': 1258840,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1258840,
 'eventLogs': [{'scoreAddress': 'cx288685685cf0a8b8f7653c00f6dc42c8406f0d5d',
   'indexed': ['PrintData(str,int,int,int)',
    'available liquidity at pool line 245',
    '0x2e141ea081ca080000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx288685685cf0a8b8f7653c00f6dc42c8406f0d5d',
   'indexed': ['PrintData(str,int,int,int)',
    'data at pool line 256',
    '0x3635dd4440d4c9f9a8',
    '0x8274de6e63dbd6b43'],
   'data': ['0x0']},
  {'scoreAddress': 'cx288685685cf0a8b8f7653c00f6dc42c8406f0d5d',
   'indexed': ['PrintData(str,int,int,int)',
    'amout of collateral needed USD pool line 268',
    '0x2519f35d8353d2db',
    '0x0'],
   'data': ['0x0']},

# Reedeming/Withdrawing USDb

In [ ]:
params ={"_amount": 10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("redeem")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

In [34]:
params ={"_reserveAddress": contracts['sample_token']['SCORE'],"_baseLTVasCollateral" :33*10**16}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("updateBaseLTVasCollateral")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx327a386a9f8076045679fc132a8baa9e270ca43b', 'stepLimit': '0x3ea80', 'timestamp': '0x5b4d764400c45', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'updateBaseLTVasCollateral', 'params': {'_reserveAddress': 'cx19be1be7b4e7750863edf966faac3f42dec21025', '_baseLTVasCollateral': '0x494654067e10000'}}, 'signature': 'g7dSUkTfVM+/zeivI3GhttfsbcabIPfbh6ZIJAm+ChEBJ8E8XhbxDdKzf1p2wzyG49tYwISwN7d3G6redmkj9wE='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x84c2bc5d2ef49ac9d74f19a54f43dcd213c10e9e5772c11e2be9ff343133df9c',
 'blockHeight': 9594840,
 'blockHash': '0xfb4f010715f54f4a8cc051bf40065e7c388d49af1d35099007e5cc779f515f9d',
 'txIndex': 4,
 'to': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
 'stepUsed': 156640,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 959180,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Repaying USDb

In [38]:
data = "{\"method\": \"repay\", \"params\": {\"amount\": 10000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 10000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xa00f366c3e3bc59fa930e6c4134a4de0ab0437250175baf3a3342ac194d7049a',
 'blockHeight': 2668,
 'blockHash': '0x51bc669d21d2e946367aea11c77091441d579d42044fe1abaf3842a27ded2a41',
 'txIndex': 1,
 'to': 'cx1592d568457a2881e4bd36117a8295ac2a000377',
 'stepUsed': 951460,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 951460,
 'eventLogs': [{'scoreAddress': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x853a0d2313c00000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 593',
    '0x2e0e918aa053e00000',
    '0x82738b02dc459515c'],
   'data': ['0x0']},
  {'scoreAddress': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26',
   'indexed': ['PrintData(str,int,int,int)',
    'Utilization rate core line 598',
    '0x216541f7accc0ec',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cxd

# Readonly methods to get data

### Getting all the addresses

In [77]:
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['addressProvider']['SCORE'])\
    .method("getAllAddresses")\
    .build()
response = icon_service.call(_call)
pprint(response)

{'collateral': {'USDb': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
                'sICX': 'cx3e9d38175d112cd5cf05a90080cce779f202ba27'},
 'oTokens': {'oICX': 'cxf6e3d64db674c0edbf732462e80e2e3352280548',
             'oUSDb': 'cx4258464e395b3ef3b73a146947eb41393e8c5873'},
 'systemContract': {'LendingPool': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
                    'LendingPoolDataProvider': 'cxa198eaf0a589adf0ab31b523d1f8912f47f6f293'}}


### Getting specific reserve data

In [53]:
#this returns the reserve data of USDb
params = {'_reserve': contracts['sicx']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if values=="0x1" or values =="0x0" or "Address" in key:
        continue
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)

pprint(response)

{'availableLiquidity': '0x0',
 'baseLTVasCollateral': 0.33,
 'borrowCumulativeIndex': 1.0,
 'borrowRate': '0x0',
 'borrowingEnabled': '0x1',
 'decimals': 1.8e-17,
 'isActive': '0x1',
 'isFreezed': '0x0',
 'lastUpdateTimestamp': '0x0',
 'liquidationBonus': 1e-17,
 'liquidationThreshold': 0.65,
 'liquidityCumulativeIndex': 1.0,
 'liquidityRate': '0x0',
 'oTokenAddress': 'cxf6e3d64db674c0edbf732462e80e2e3352280548',
 'reserveAddress': 'cx3e9d38175d112cd5cf05a90080cce779f202ba27',
 'totalBorrows': '0x0',
 'totalLiquidity': '0x0',
 'usageAsCollateralEnabled': '0x1'}


# Reserve Account Data

In [74]:
#this returns the reserve data of USDb

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveAccountData")\
    .build()
response = icon_service.call(_call)
# print(response)
for key,values in response.items():
    response[key]=int(values,0)/10**18
        

pprint(response)

{'availableLiquidityBalanceUSD': 22999.99999792432,
 'totalBorrowsBalanceUSD': 880.3978710152415,
 'totalCollateralBalanceUSD': 23880.39786893956,
 'totalLiquidityBalanceUSD': 23880.39786893956}


### Getting data of all reserves

In [57]:
#this returns the reserve data of all the reserves
params = {'_reserve': contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getAllReserveData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if v=="0x1" or v =="0x0" or "Address" in k:
            continue
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'availableLiquidity': 22999.99999792432,
          'baseLTVasCollateral': 0.33,
          'borrowCumulativeIndex': 1.0000132994517619,
          'borrowRate': 0.004301146300639056,
          'borrowingEnabled': '0x1',
          'decimals': 1.8e-17,
          'isActive': '0x1',
          'isFreezed': '0x0',
          'lastUpdateTimestamp': 1606153045411763,
          'liquidationBonus': 1e-17,
          'liquidationThreshold': 0.65,
          'liquidityCumulativeIndex': 1.0000000468647359,
          'liquidityRate': 0.000142713201853293,
          'oTokenAddress': 'cx4258464e395b3ef3b73a146947eb41393e8c5873',
          'reserveAddress': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
          'totalBorrows': 880.3978710152415,
          'totalLiquidity': 23880.39786893956,
          'usageAsCollateralEnabled': '0x1'},
 'icx': {'availableLiquidity': '0x0',
         'baseLTVasCollateral': 0.33,
         'borrowCumulativeIndex': 1.0,
         'borrowRate': '0x0',
         'borrowi

### Getting user account data

In [58]:
#returns the user data of all the reserves combined
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserAccountData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'currentLiquidationThreshold': 0.65,
 'currentLtv': 0.33,
 'healthFactor': -1e-18,
 'healthFactorBelowThreshold': 0.0,
 'totalBorrowBalanceUSD': 0.0,
 'totalCollateralBalanceUSD': 5000.001615115605,
 'totalFeesUSD': 0.0,
 'totalLiquidityBalanceUSD': 5000.001615115605}


In [53]:
#returns the user data of all the reserves combined
params = {'_reserve': contracts['sample_token']['SCORE'],'_amount':10*10**18,"_fee":1,'_userCurrentBorrowBalanceUSD':0,'_userCurrentFeesUSD':0,'_userCurrentLtv':5*10**17}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("calculateCollateralNeededUSD")\
    .params(params)\
    .build()
response = icon_service.call(_call)
response = int(response,0)/10**18
# for key,values in response.items():
#     if key!="lastUpdateTimestamp":
#         response[key]=int(values,0)/10**18
#     else:
#         response[key]=int(values,0)
print(response)

20.0


### Getting user reserve data

In [59]:
#returns the data of a specific user for a specific reserve
#this returns the data of user for USDb reserve
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' ,"_reserve":contracts['sicx']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'borrowRate': 0.0,
 'currentBorrowBalance': 0.0,
 'currentBorrowBalanceUSD': 0.0,
 'currentOTokenBalance': 0.0,
 'currentOTokenBalanceUSD': 0.0,
 'lastUpdateTimestamp': 0,
 'liquidityRate': 0.0,
 'originationFee': 0.0,
 'principalBorrowBalance': 0.0,
 'principalBorrowBalanceUSD': 0.0,
 'useAsCollateral': 0.0,
 'userBorrowCumulativeIndex': 0.0}


### Getting the data of user for all reserves

In [60]:
#returns the data of a specific user for all reserves

params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserAllReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'borrowRate': 0.004301146300639056,
          'currentBorrowBalance': 0.0,
          'currentBorrowBalanceUSD': 0.0,
          'currentOTokenBalance': 5000.00161599806,
          'currentOTokenBalanceUSD': 5000.00161599806,
          'lastUpdateTimestamp': 0,
          'liquidityRate': 0.000142713201853293,
          'originationFee': 0.0,
          'principalBorrowBalance': 0.0,
          'principalBorrowBalanceUSD': 0.0,
          'useAsCollateral': 1e-18,
          'userBorrowCumulativeIndex': 0.0},
 'icx': {'borrowRate': 0.0,
         'currentBorrowBalance': 0.0,
         'currentBorrowBalanceUSD': 0.0,
         'currentOTokenBalance': 0.0,
         'currentOTokenBalanceUSD': 0.0,
         'lastUpdateTimestamp': 0,
         'liquidityRate': 0.0,
         'originationFee': 0.0,
         'principalBorrowBalance': 0.0,
         'principalBorrowBalanceUSD': 0.0,
         'useAsCollateral': 0.0,
         'userBorrowCumulativeIndex': 0.0}}


### Get reserve configurations


In [61]:
#returns the configurations of a specific reserve
params = {"_reserve":contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveConfigurationData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    response[key]=int(values,0)
pprint(response)


{'baseLTVasCollateral': 330000000000000000,
 'borrowingEnabled': 1,
 'decimals': 18,
 'isActive': 1,
 'liquidationBonus': 10,
 'liquidationThreshold': 650000000000000000,
 'usageAsCollateralEnabled': 1}


### Get all reserves configurations

In [62]:
#returns the configurations of all the   reserves

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getAllReserveConfigurationData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        response[key][k]=int(v,0)
pprint(response)


{'USDb': {'baseLTVasCollateral': 330000000000000000,
          'borrowingEnabled': 1,
          'decimals': 18,
          'isActive': 1,
          'liquidationBonus': 10,
          'liquidationThreshold': 650000000000000000,
          'usageAsCollateralEnabled': 1},
 'icx': {'baseLTVasCollateral': 330000000000000000,
         'borrowingEnabled': 1,
         'decimals': 18,
         'isActive': 1,
         'liquidationBonus': 10,
         'liquidationThreshold': 650000000000000000,
         'usageAsCollateralEnabled': 1}}


## Realtime USDb balance of user(with accrued interest)

In [33]:

params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']['SCORE'])\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)

print(int(response,0))

0


In [20]:

# params = {'_owner': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getOracleAddress")\
    .build()
response = icon_service.call(_call)
print(response)

cx8397f621adac63d786c24cfdf920e683ae320dfd


In [ ]:
995000000570919367819
995000000564440311889
894000001574533095848
894000000615847612243
894000001574533095848
795000001589272371715
795000001731223349511